
TRABAJO PRACTICO MATE 3 SOBRE LA PREDICCION DEL TIPO DE VINO (WHITE OR RED(BLANCO O NEGRO))


Al principio queria realizar uno que prediga la calidad del vino por los datos, pero utilizar el coeficiente de relacion me di cuenta que al parecer la calidad entra por otro lado y no se puede predecir solo con estadistica, pero hay algunos datos que poseen coorelacion moderada y fuerte con la que voy a intentar predecir el tipo de vino

EXTRAIGO EL DATASET A UN DATAFRAME


In [27]:
import numpy as np
import pandas as pd

# Extraigo los valores del csv en mi variabla data delimitando los datos con la "," y los numeros decimales con el "."
data = pd.read_csv("WineQuality.csv", delimiter=",", decimal=".")

PRIMERO CAMBIO EL DATAFRAME PARA QUE EN VEZ DE QUE EN LA COLUMNA INDIQUE WHITE Y RED SEA WHITE "1" Y RED "0" PARA PODER REALIZAR LOS CALCULOS DE MEJOR MANERA


In [28]:
# Reemplazar los valores en la columna "type"
data['type'] = data['type'].replace({'white': 1, 'red': 0})

# Verificar los cambios
print(data["type"])  # Imprimo la columna para verificar

0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: type, Length: 6497, dtype: int64


C:\Users\loren\AppData\Local\Temp\ipykernel_29884\2130396368.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['type'] = data['type'].replace({'white': 1, 'red': 0})


BUSCO CORRELACION DE LAS CARACTERISTICAS CON LA CALIDAD DEL VINO
(LA IDEA ORIGINAL ERA CALCULAR LA CALIDAD DEL VINO PERO COMO LA CORRELACION DE LA MISMA CON LOS DATOS DE ENTRADA ERAN MUY CERCANOS A 0 AHORA VOY A PREDECIR DE QUE COLOR ES EL VINO)

In [29]:
# Imprimir correlaciones entre variables
correlations = data.corr(method='pearson')
print(correlations)

                          type  fixed acidity  volatile acidity  citric acid  \
type                  1.000000      -0.487241         -0.652875     0.187126   
fixed acidity        -0.487241       1.000000          0.220172     0.323736   
volatile acidity     -0.652875       0.220172          1.000000    -0.378061   
citric acid           0.187126       0.323736         -0.378061     1.000000   
residual sugar        0.349000      -0.112319         -0.196702     0.142486   
chlorides            -0.512622       0.298421          0.377167     0.039315   
free sulfur dioxide   0.471644      -0.283317         -0.353230     0.133437   
total sulfur dioxide  0.700357      -0.329747         -0.414928     0.195218   
density              -0.390645       0.459204          0.271193     0.096320   
pH                   -0.328727      -0.251814          0.260660    -0.328689   
sulphates            -0.486916       0.300380          0.225476     0.057613   
alcohol               0.032970      -0.0

UTILIZO LOS DATOS EN QUE POSEAN MAS DE 0.47 DE COORELACION CON TYPE PARA CALCULAR CUAL ES LA PRECISION

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Definir características y etiquetas
features = ["volatile acidity", "chlorides",
            "total sulfur dioxide", "sulphates"]  # Seleccionar las características relevantes
X = data[features].values
y = data['type'].values

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=1/3, random_state=42)

# Asegurar que los datos tengan la forma correcta
X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape(1, -1)
y_test = y_test.reshape(1, -1)

# Número de registros de entrenamiento
n = X_train.shape[1]

# Inicializar los pesos y sesgos de la red neuronal
np.random.seed(42)
w_hidden = np.random.rand(3, X_train.shape[0])
w_output = np.random.rand(1, 3)

b_hidden = np.random.rand(3, 1)
b_output = np.random.rand(1, 1)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Función para propagar hacia adelante (forward propagation)
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2

# Calcular precisión
test_predictions = forward_prop(X_test)[3]  # Solo nos interesa la capa de salida, A2
test_comparisons = np.equal((test_predictions >= 0.5).astype(int).flatten(), y_test.flatten())
accuracy = np.sum(test_comparisons.astype(int)) / X_test.shape[1]
print("Accuracy:", accuracy)

Accuracy: 0.7553093259464451


Realizo el descenso de gradiente


In [31]:
# Definir características y etiquetas
features = ["volatile acidity", "chlorides", 
            "total sulfur dioxide", "sulphates"]  # Seleccionar las características relevantes
X = data[features].values
y = data['type'].values

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=1/3, random_state=42)

# Asegurar que los datos tengan la forma correcta
X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape(1, -1)
y_test = y_test.reshape(1, -1)

# Número de registros de entrenamiento
n = X_train.shape[1]
print(n)

# Inicializar los pesos y sesgos de la red neuronal
np.random.seed(42)
w_hidden = np.random.rand(3, X_train.shape[0])
w_output = np.random.rand(1, 3)

b_hidden = np.random.rand(3, 1)
b_output = np.random.rand(1, 1)

# Funciones de activación
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Función para propagar hacia adelante (forward propagation)
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2

L = 0.05  # La tasa de aprendizaje

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = logistic(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

# Ejecutar descenso de gradiente
for i in range(100_000):
    # seleccionar aleatoriamente un conjunto de datos de entrenamiento
    indices = np.random.choice(n, 1, replace=False)  # seleccionar 32 índices aleatorios
    X_batch = X_train[:, indices]  # No es necesario transponer X_batch
    Y_batch = y_train[:, indices]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2 = forward_prop(X_batch)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_batch, Y_batch)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2

4331


UTILIZO LA RED NEURONAL PARA REALIZAR PREDICCIONES

In [36]:
# Definir características de entrada para los nuevos datos
new_features = ["volatile acidity", "chlorides", "total sulfur dioxide", "sulphates"]

# Seleccionar características relevantes de los nuevos datos
new_data = {
    "volatile acidity": [0.27, 0.3, 0.28, 0.725],# 0.31, 0.28],
    "chlorides": [0.045, 0.049, 0.05, 0.073],# 0.067, 0.05],
    "total sulfur dioxide": [170, 132, 97, 35],# 190, 97],
    "sulphates": [0.45, 0.49, 0.44, 0.82]#, 0.47, 0.44]
# [0.27, 0.045, 170, 0.45]
# [0.3, 0.049, 132, 0.49]
# [0.28, 0.05, 97, 0.44]
# [0.725, 0.073, 35, 0.82] COLUMNAS DE LOS EJEMPLOS A PREDECIR
}

# Convertir el diccionario de datos a un DataFrame
new_df = pd.DataFrame(new_data)

# Escalar las características de los nuevos datos utilizando el mismo escalador
X_new_scaled = scaler.transform(new_df.values)

# List to store predictions
predictions = []

# Iterar sobre los nuevos datos y hacer predicciones de uno en uno
for i in range(X_new_scaled.shape[1]):
    # Obtener un ejemplo a la vez
    X_example = X_new_scaled[:, i].reshape(-1, 1)
    
    # Propagar hacia adelante el ejemplo a través de la red neuronal
    Z1_new, A1_new, Z2_new, A2_new = forward_prop(X_example)
    
    # Obtener la predicción (0 para vino tinto y 1 para vino blanco) usando un umbral de 0.5
    prediction = int(A2_new > 0.5)
    
    # Agregar la predicción a la lista de predicciones
    predictions.append(prediction)

# Mostrar las predicciones
print(predictions)

[0, 0, 0, 0]


C:\Users\loren\AppData\Local\Temp\ipykernel_29884\346542742.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction = int(A2_new > 0.5)
